In [ ]:
#=======================================================================================================================================================================================
# Script 1: Install Required Packages
#=======================================================================================================================================================================================
import os
os.system("pip install loguru colorama rich tqdm pillow requests colorthief python-dotenv ipython ipywidgets")

In [ ]:
#=======================================================================================================================================================================================
# Script 2: Rename And Delete Duplicate Images
#=======================================================================================================================================================================================
from colorama import init,Fore,Style
from loguru import logger
import hashlib
import os
NAME= "picture"
init(autoreset=True)
def hash_file(file_path):
    hasher=hashlib.md5()
    with open(file_path,"rb") as f:
        while chunk:=f.read(8192):
            hasher.update(chunk)
    return hasher.hexdigest()
def rename_and_remove_duplicates(base_folder):
    file_hashes={}
    try:
        for root,_,files in os.walk(base_folder):
            folder_counter=1
            files=[f for f in files if f.lower().endswith(".jpg")]
            for file in files:
                file_path=os.path.join(root,file)
                file_hash=hash_file(file_path)
                if file_hash in file_hashes:
                    print(f"{Fore.RED}INFO:{Style.RESET_ALL} Duplicate found and deleted: {file_path} (duplicate of {file_hashes[file_hash]})")
                    os.remove(file_path)
                else:
                    file_hashes[file_hash]=file_path
                    new_name=f"{NAME} ({folder_counter}).jpg"
                    new_path=os.path.join(root,new_name)
                    while os.path.exists(new_path):
                        folder_counter+=1
                        new_name=f"{NAME} ({folder_counter}).jpg"
                        new_path=os.path.join(root,new_name)
                    os.rename(file_path,new_path)
                    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} Done: {new_name} (in) {root}")
                    folder_counter+=1
        print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} Renaming and duplicate deletion process completed.")
    except Exception as e:
        logger.error(f"An error occurred during rename and duplicate deletion process: {e}")
if __name__=="__main__":
    base_folder_path=os.path.join("scripts","mixed")
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} Starting renaming and duplicate deletion process...")
    rename_and_remove_duplicates(base_folder_path)
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} Process completed.")

In [ ]:
#=======================================================================================================================================================================================
# Script 3: Rename and Delete Duplicates
#=======================================================================================================================================================================================
from colorama import init, Fore, Style
from loguru import logger
init(autoreset=True)
import os
import re
def rename_files(base_directory):
    for root, dirs, files in os.walk(base_directory):
        file_groups = {}
        for file in files:
            if file.endswith(".jpg"):
                base_name = re.sub(r" \(\d+\)\.jpg$","",file)
                base_name = re.sub(r" \d+\.jpg$","",base_name)
                if base_name not in file_groups:
                    file_groups[base_name] = []
                file_groups[base_name].append(file)
        for base_name, group in file_groups.items():
            group.sort()
            for i, file in enumerate(group,start=1):
                old_path = os.path.join(root,file)
                new_file_name = f"{base_name} ({i}).jpg"
                new_path = os.path.join(root,new_file_name)
                os.rename(old_path,new_path)
                print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} Renamed: {old_path} -> {new_path}")
rename_files(os.path.join("source","input"))

In [ ]:
#=======================================================================================================================================================================================
# Script 4: Image Enhancement And Upscaling
#=======================================================================================================================================================================================
from PIL import Image, ImageEnhance, ImageFile
from colorama import init, Fore, Style
from loguru import logger
from tqdm import tqdm
init(autoreset=True)
import subprocess
import os
import re
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True
def load_images(input_path):
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🔍 Searching for images...")
    if os.path.isdir(input_path):
        images = []
        for file_name in os.listdir(input_path):
            file_path = os.path.join(input_path,file_name)
            if file_path.lower().endswith((".png",".jpg",".jpeg",".webp")):
                images.append(file_path)
        images.sort(key=lambda x: int(re.search(r'(\d+)',x).group()) if re.search(r'(\d+)',x) else 0)
        return images
    else:
        return [input_path]
def apply_filters(image,sharpness,saturation,brightness,contrast):
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🎨 Applying image filters:")
    logger.debug(f"Sharpness: {sharpness}% | Saturation: {saturation}% | Brightness: {brightness}% | Contrast: {contrast}%")
    enhancer = ImageEnhance.Sharpness(image)
    image = enhancer.enhance(1 + sharpness/100)
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(1 + saturation/100)
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1 + brightness/100)
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1 + contrast/100)
    return image
def save_temp_image(image,original_image_path):
    temp_dir = os.path.join(os.path.dirname(original_image_path),"temp")
    os.makedirs(temp_dir,exist_ok=True)
    temp_image_path = os.path.join(temp_dir,f"temp_{os.path.basename(original_image_path)}")
    image.save(temp_image_path)
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 💾 Temporary image saved: {temp_image_path}")
    return temp_image_path
def run_upscale_command(input_path,output_path,model_name):
    upscaling_command = [
        os.path.join("include","real-esrgan","engine.exe"),
        "-n",model_name,"-o",output_path,"-i",input_path,
        "-m",os.path.join("include","real-esrgan","models"),
        "-f",input_path.split(".")[-1],"-s","4", "-x", "-t", "256",
    ]
    progress_pattern = re.compile(r"(\d+\.\d+)%")
    process = subprocess.Popen(upscaling_command,shell=True,bufsize=0,text=True,stderr=subprocess.STDOUT,universal_newlines=True,stdout=subprocess.PIPE)
    with tqdm(total=100,desc="INFO: 🔬 Upscaling Progress",unit="%") as pbar:
        last_percentage = 0
        while True:
            line = process.stdout.readline()
            if not line and process.poll() is not None:
                break
            if line.strip():
                match = progress_pattern.search(line)
                if match:
                    try:
                        current_percentage = float(match.group(1))
                        if current_percentage > last_percentage:
                            pbar.update(current_percentage - last_percentage)
                            last_percentage = current_percentage
                    except ValueError:
                        pass
        if last_percentage < 100:
            pbar.update(100 - last_percentage)
    process.wait()
    if process.returncode != 0:
        logger.error("Error during the upscaling process!")
        raise Exception("Error during the upscaling process.")
def save_final_image(output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 💾 Final images will be saved to: {output_path}")
def cleanup(temp_image_path):
    if os.path.exists(temp_image_path):
        os.remove(temp_image_path)
        print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🗑️ Temporary image removed")
    temp_dir = os.path.dirname(temp_image_path)
    if not os.listdir(temp_dir):
        os.rmdir(temp_dir)
        print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🧹 Temporary directory cleaned up")
def inference(input_path,sharpness,saturation,brightness,contrast,image_anime_style=False,double_upscale_4x_to_16x=False):
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🎨 Starting Image Enhancement and Upscaling 🚀")
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🖼️  Image Processing Pipeline Initiated 🚀")
    image_paths = load_images(input_path)
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🖼️  Found {len(image_paths)} image(s) to process")
    for image_path in image_paths:
        print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🔧 Processing image: {os.path.basename(image_path)}")
        image = Image.open(image_path)
        logger.debug(f"Original image size: {image.size}")
        image = apply_filters(image,sharpness,saturation,brightness,contrast)
        print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} ✨ Filters applied successfully")
        temp_image_path = save_temp_image(image,image_path)
        print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🚀 Upscaling image...")
        original_ext = os.path.splitext(input_path)[1]
        final_image_name = os.path.basename(temp_image_path).replace("temp_","")
        final_image_name_without_ext = os.path.splitext(final_image_name)[0]
        output_path = os.path.join(os.path.dirname(image_path),"output",f"{final_image_name_without_ext}{original_ext}")
        model_name = "realesrgan-x4plus-anime" if image_anime_style else "realesrgan-x4plus"
        print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🤖 Using model: {model_name}")
        print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 📊 Upscaling mode: {'Double Upscale (4x to 16x)' if double_upscale_4x_to_16x else 'Single Upscale (4x)'}")
        if double_upscale_4x_to_16x:
            intermediate_output_path = temp_image_path.replace(".png","_4x.png")
            run_upscale_command(temp_image_path,intermediate_output_path,model_name)
            final_output_path = intermediate_output_path.replace("_4x","_16x")
            run_upscale_command(intermediate_output_path,final_output_path,model_name)
            final_image_name = os.path.splitext(os.path.basename(final_output_path).replace("temp_",""))[0]
            output_path = os.path.join(os.path.dirname(image_path),"output",f"{final_image_name}{original_ext}")
            os.rename(final_output_path,output_path)
        else:
            run_upscale_command(temp_image_path,output_path,model_name)
        save_final_image(os.path.dirname(image_path))
        cleanup(temp_image_path)
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} ✅ Image Processing Completed Successfully!")
    print(f"{Fore.GREEN}INFO:{Style.RESET_ALL} 🎉 Image Processing Script Finished!")
if __name__ == "__main__":
    main_directory = os.path.join("source","input")
    folder_names = [name for name in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory,name))]
    for name in folder_names:
        inference(
            image_anime_style=("Anime" in os.path.join(main_directory,name)),
            brightness=10, saturation=10, sharpness=80, contrast=10,
            input_path=os.path.join(main_directory,name),
            double_upscale_4x_to_16x=False
        )